In [ ]:
# ============================================================
# CONFIGURACIÓN E IMPORTS
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path
import sys
import os

# Agregar la raíz del proyecto al path
ROOT = Path().resolve().parent
sys.path.insert(0, str(ROOT))

# Importar configuración centralizada
from config import PATHS, CONFIG

# ============= CONFIGURACIÓN USANDO CONFIG.PY =============

# Usar rutas centralizadas
ARCHIVO_EXCEL = PATHS.RAW / "Formulario.xlsx"
CARPETA_SALIDA = PATHS.FEATURES

print(f"\n Configuración:")
print(f"  - Archivo Excel: {ARCHIVO_EXCEL}")
print(f"  - Carpeta salida: {CARPETA_SALIDA}")

In [ ]:
def procesar_formulario_psicometrico():
    """
    Procesa formulario con escalas UCLA y DASS-21
    """
    
    print("="*80)
    print("PROCESAMIENTO DE DATOS PSICOMÉTRICOS")
    print("="*80)
    
    # Crear carpeta de salida
    if not os.path.exists(CARPETA_SALIDA):
        os.makedirs(CARPETA_SALIDA)
        print(f"✓ Carpeta '{CARPETA_SALIDA}/' creada")
    
    # ========================================
    # 1. CARGAR ARCHIVO EXCEL
    # ========================================
    
    print(f"\n{'='*80}")
    print("CARGANDO FORMULARIO")
    print(f"{'='*80}")
    
    if not os.path.exists(ARCHIVO_EXCEL):
        print(f"Error: No se encuentra el archivo '{ARCHIVO_EXCEL}'")
        return None
    
    try:
        df = pd.read_excel(ARCHIVO_EXCEL)
        print(f"✓ Archivo cargado: {len(df)} respuestas")
    except Exception as e:
        print(f"Error al leer archivo: {e}")
        return None
    
    print(f"\nColumnas encontradas ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i}. {col}")
    
    # ========================================
    # 2. IDENTIFICAR Y RENOMBRAR COLUMNAS
    # ========================================
    
    print(f"\n{'='*80}")
    print("IDENTIFICANDO COLUMNAS")
    print(f"{'='*80}")
    
    # Mapeo de nombres de columnas (basado en la imagen)
    renombrar = {}
    
    # Buscar columnas demográficas
    for col in df.columns:
        col_lower = col.lower()
        if 'marca temporal' in col_lower or 'timestamp' in col_lower:
            renombrar[col] = 'timestamp'
        elif 'seudonimo' in col_lower or 'seudónimo' in col_lower:
            renombrar[col] = 'id_participante'
        elif 'edad' in col_lower:
            renombrar[col] = 'edad'
        elif 'género' in col_lower or 'genero' in col_lower:
            renombrar[col] = 'genero'
        elif 'año académico' in col_lower or 'ano academico' in col_lower:
            renombrar[col] = 'ano_academico'
        elif 'actividades extracurriculares' in col_lower:
            renombrar[col] = 'actividades_extra'
        elif 'miembro activo' in col_lower and 'whatsapp' in col_lower:
            renombrar[col] = 'miembro_wa'
    
    df = df.rename(columns=renombrar)
    print(f"✓ {len(renombrar)} columnas demográficas identificadas")
    
    # ========================================
    # 3. IDENTIFICAR COLUMNAS UCLA
    # ========================================
    
    print(f"\n{'='*80}")
    print("PROCESANDO ESCALA UCLA (SOLEDAD)")
    print(f"{'='*80}")
    
    # Buscar columnas UCLA - identificar por palabras clave en el texto
    columnas_ucla = []
    palabras_clave_ucla = [
        'sintonía', 'compañía', 'recurrir', 'solo', 'grupo de amigos',
        'en común', 'relación cercana', 'intereses', 'extrovertido',
        'cercano', 'excluido', 'significativas', 'conoce', 'aislado',
        'encontrar compañía', 'entienden', 'tímido', 'a tu alrededor',
        'puedes hablar', 'puedes recurrir'
    ]
    
    for col in df.columns:
        col_lower = col.lower()
        # Verificar si la columna contiene palabras clave de UCLA
        if any(palabra in col_lower for palabra in palabras_clave_ucla):
            columnas_ucla.append(col)
    
    # Si no funcionó, usar método alternativo: las primeras 20 preguntas numeradas
    if len(columnas_ucla) != 20:
        columnas_ucla = []
        contador = 0
        for i, col in enumerate(df.columns):
            # Buscar columnas que empiezan con "número." después de las demográficas
            if any(col.strip().startswith(f"{num}. ") for num in range(1, 21)):
                columnas_ucla.append(col)
                contador += 1
                if contador == 20:
                    break
    
    print(f"✓ {len(columnas_ucla)} ítems UCLA identificados")
    
    if len(columnas_ucla) != 20:
        print(f"Advertencia: Se esperaban 20 ítems UCLA, se encontraron {len(columnas_ucla)}")
        print(f"    Primeras columnas identificadas:")
        for i, col in enumerate(columnas_ucla[:5], 1):
            print(f"      {i}. {col[:80]}...")
    
    # Mapeo de respuestas UCLA
    # Primero, verificar qué respuestas únicas hay
    respuestas_unicas = set()
    for col in columnas_ucla:
        respuestas_unicas.update(df[col].dropna().unique())
    
    print(f"\nRespuestas únicas encontradas en UCLA:")
    for resp in sorted(respuestas_unicas, key=lambda x: str(x)):
        print(f"  - '{resp}'")
    
    # Las respuestas ya vienen como números en string, solo convertir
    mapeo_ucla = {
        '1': 1, 1: 1,
        '2': 2, 2: 2,
        '3': 3, 3: 3,
        '4': 4, 4: 4,
        # Por si acaso vienen como texto
        'Nunca': 1,
        'Rara vez': 2,
        'Raramente': 2,
        'A veces': 3,
        'Algunas veces': 3,
        'Siempre': 4,
        'A menudo': 4,
        'Casi nunca': 1,
        'Casi siempre': 4,
        'Frecuentemente': 4,
        'Ocasionalmente': 3
    }
    
    # Aplicar mapeo a columnas UCLA
    valores_no_mapeados = 0
    for col in columnas_ucla:
        df[col] = df[col].map(mapeo_ucla)
        no_mapeados = df[col].isna().sum()
        if no_mapeados > 0:
            valores_no_mapeados += no_mapeados
    
    if valores_no_mapeados > 0:
        print(f"Total de valores no mapeados en UCLA: {valores_no_mapeados}")
    else:
        print(f"✓ Todas las respuestas UCLA mapeadas correctamente")
    
    # Calcular puntuación total UCLA
    df['ucla_total'] = df[columnas_ucla].sum(axis=1)
    
    print(f"\nEstadísticas UCLA:")
    print(f"  Media: {df['ucla_total'].mean():.2f}")
    print(f"  Mediana: {df['ucla_total'].median():.2f}")
    print(f"  Mínimo: {df['ucla_total'].min():.0f}")
    print(f"  Máximo: {df['ucla_total'].max():.0f}")
    print(f"  Desviación estándar: {df['ucla_total'].std():.2f}")
    
    # Clasificación de soledad (basada en literatura)
    def clasificar_ucla(score):
        if pd.isna(score):
            return 'Sin datos'
        elif score <= 28:
            return 'Baja soledad'
        elif score <= 43:
            return 'Soledad moderada'
        else:
            return 'Alta soledad'
    
    df['ucla_categoria'] = df['ucla_total'].apply(clasificar_ucla)
    
    print(f"\nDistribución de soledad:")
    print(df['ucla_categoria'].value_counts().to_string())
    
    # ========================================
    # 4. IDENTIFICAR COLUMNAS DASS-21
    # ========================================
    
    print(f"\n{'='*80}")
    print("PROCESANDO ESCALA DASS-21")
    print(f"{'='*80}")
    
    # Buscar columnas DASS-21 - identificar por palabras clave
    columnas_dass = []
    palabras_clave_dass = [
        'tensión', 'boca seca', 'emoción positiva', 'dificultades para respirar',
        'iniciativa', 'reaccionar exageradamente', 'temblores', 'gastando energía',
        'presa del pánico', 'ilusionara', 'agitado', 'relajarme', 'desanimado',
        'tolerado', 'borde del pánico', 'entusiasmarme', 'no valía',
        'enfadado', 'corazón', 'asustado', 'vida no tenía sentido'
    ]
    
    for col in df.columns:
        col_lower = col.lower()
        # Verificar si la columna contiene palabras clave de DASS
        if any(palabra in col_lower for palabra in palabras_clave_dass):
            columnas_dass.append(col)
    
    # Si no funcionó, buscar después de las UCLA
    if len(columnas_dass) != 21:
        columnas_dass = []
        # Saltar las primeras 7 columnas (demográficas) y las 20 de UCLA
        inicio_dass = 7 + 20
        for i, col in enumerate(df.columns):
            if i >= inicio_dass:
                # Buscar columnas que empiezan con "número."
                if any(col.strip().startswith(f"{num}. ") for num in range(1, 22)):
                    columnas_dass.append(col)
                    if len(columnas_dass) == 21:
                        break
    
    print(f"✓ {len(columnas_dass)} ítems DASS-21 identificados")
    
    if len(columnas_dass) != 21:
        print(f"Advertencia: Se esperaban 21 ítems DASS, se encontraron {len(columnas_dass)}")
        print(f"    Primeras columnas identificadas:")
        for i, col in enumerate(columnas_dass[:5], 1):
            print(f"      {i}. {col[:80]}...")
    
    # Mapeo de respuestas DASS-21
    # Primero, verificar qué respuestas únicas hay
    respuestas_unicas_dass = set()
    for col in columnas_dass:
        respuestas_unicas_dass.update(df[col].dropna().unique())
    
    print(f"\nRespuestas únicas encontradas en DASS-21:")
    for resp in sorted(respuestas_unicas_dass, key=lambda x: str(x)):
        print(f"  - '{resp}'")
    
    # Las respuestas ya vienen como números en string, solo convertir
    mapeo_dass = {
        '0': 0, 0: 0,
        '1': 1, 1: 1,
        '2': 2, 2: 2,
        '3': 3, 3: 3,
        # Por si acaso vienen como texto
        'No me aplicó en absoluto': 0,
        'Me aplicó un poco, o durante parte del tiempo': 1,
        'Me aplicó bastante, o durante una buena parte del tiempo': 2,
        'Me aplicó mucho, o la mayor parte del tiempo': 3,
        'Nunca': 0,
        'A veces': 1,
        'A menudo': 2,
        'Casi siempre': 3,
        'Nada': 0,
        'Un poco': 1,
        'Bastante': 2,
        'Mucho': 3,
        'No': 0,
        'Sí, un poco': 1,
        'Sí, bastante': 2,
        'Sí, mucho': 3
    }
    
    # Aplicar mapeo a columnas DASS
    valores_no_mapeados_dass = 0
    for col in columnas_dass:
        df[col] = df[col].map(mapeo_dass)
        no_mapeados = df[col].isna().sum()
        if no_mapeados > 0:
            valores_no_mapeados_dass += no_mapeados
    
    if valores_no_mapeados_dass > 0:
        print(f"Total de valores no mapeados en DASS-21: {valores_no_mapeados_dass}")
    else:
        print(f"✓ Todas las respuestas DASS-21 mapeadas correctamente")
    
    # DASS-21 tiene 3 subescalas (7 ítems cada una)
    # Los ítems se distribuyen así:
    # Depresión: 3, 5, 10, 13, 16, 17, 21 (posiciones en el cuestionario completo)
    # Ansiedad: 2, 4, 7, 9, 15, 19, 20
    # Estrés: 1, 6, 8, 11, 12, 14, 18
    
    if len(columnas_dass) == 21:
        # Índices 0-based para las columnas
        items_depresion = [2, 4, 9, 12, 15, 16, 20]  # Posiciones en el array
        items_ansiedad = [1, 3, 6, 8, 14, 18, 19]
        items_estres = [0, 5, 7, 10, 11, 13, 17]
        
        # Extraer valores específicos
        df['dass_depresion_raw'] = sum(df[columnas_dass[i]] for i in items_depresion)
        df['dass_ansiedad_raw'] = sum(df[columnas_dass[i]] for i in items_ansiedad)
        df['dass_estres_raw'] = sum(df[columnas_dass[i]] for i in items_estres)
        
        # Multiplicar por 2 para equiparar a DASS-42
        df['dass_depresion'] = df['dass_depresion_raw'] * 2
        df['dass_ansiedad'] = df['dass_ansiedad_raw'] * 2
        df['dass_estres'] = df['dass_estres_raw'] * 2
        
        print(f"\nEstadísticas DASS-21:")
        print(f"\nDepresión:")
        print(f"  Media: {df['dass_depresion'].mean():.2f}")
        print(f"  Mediana: {df['dass_depresion'].median():.2f}")
        print(f"  Rango: {df['dass_depresion'].min():.0f} - {df['dass_depresion'].max():.0f}")
        
        print(f"\nAnsiedad:")
        print(f"  Media: {df['dass_ansiedad'].mean():.2f}")
        print(f"  Mediana: {df['dass_ansiedad'].median():.2f}")
        print(f"  Rango: {df['dass_ansiedad'].min():.0f} - {df['dass_ansiedad'].max():.0f}")
        
        print(f"\nEstrés:")
        print(f"  Media: {df['dass_estres'].mean():.2f}")
        print(f"  Mediana: {df['dass_estres'].median():.2f}")
        print(f"  Rango: {df['dass_estres'].min():.0f} - {df['dass_estres'].max():.0f}")
        
        # Clasificaciones DASS-21 (basadas en literatura)
        def clasificar_depresion(score):
            if pd.isna(score):
                return 'Sin datos'
            elif score <= 9:
                return 'Normal'
            elif score <= 13:
                return 'Leve'
            elif score <= 20:
                return 'Moderada'
            elif score <= 27:
                return 'Severa'
            else:
                return 'Extremadamente severa'
        
        def clasificar_ansiedad(score):
            if pd.isna(score):
                return 'Sin datos'
            elif score <= 7:
                return 'Normal'
            elif score <= 9:
                return 'Leve'
            elif score <= 14:
                return 'Moderada'
            elif score <= 19:
                return 'Severa'
            else:
                return 'Extremadamente severa'
        
        def clasificar_estres(score):
            if pd.isna(score):
                return 'Sin datos'
            elif score <= 14:
                return 'Normal'
            elif score <= 18:
                return 'Leve'
            elif score <= 25:
                return 'Moderado'
            elif score <= 33:
                return 'Severo'
            else:
                return 'Extremadamente severo'
        
        df['dass_depresion_cat'] = df['dass_depresion'].apply(clasificar_depresion)
        df['dass_ansiedad_cat'] = df['dass_ansiedad'].apply(clasificar_ansiedad)
        df['dass_estres_cat'] = df['dass_estres'].apply(clasificar_estres)
        
        print(f"\nDistribución de depresión:")
        print(df['dass_depresion_cat'].value_counts().to_string())
        
        print(f"\nDistribución de ansiedad:")
        print(df['dass_ansiedad_cat'].value_counts().to_string())
        
        print(f"\nDistribución de estrés:")
        print(df['dass_estres_cat'].value_counts().to_string())
    
    # ========================================
    # 5. FILTRAR PARTICIPANTES EXCLUIDOS
    # ========================================
    
    print(f"\n{'='*80}")
    print("APLICANDO CRITERIOS DE EXCLUSIÓN")
    print(f"{'='*80}")
    
    # Participantes a excluir (< 10 publicaciones o datos incompletos)
    PARTICIPANTES_EXCLUIDOS = ['EST005', 'EST006', 'EST022']
    
    n_antes = len(df)
    df = df[~df['id_participante'].isin(PARTICIPANTES_EXCLUIDOS)]
    n_excluidos = n_antes - len(df)
    
    if n_excluidos > 0:
        print(f"✓ Excluidos {n_excluidos} participantes:")
        for pid in PARTICIPANTES_EXCLUIDOS:
            if pid in df[~df['id_participante'].isin(PARTICIPANTES_EXCLUIDOS)].index:
                continue
            print(f"  - {pid} (< 10 publicaciones o datos incompletos)")
    
    print(f"\nMuestra final: n={len(df)} participantes")
    
    # ========================================
    # 6. CREAR DATASET LIMPIO
    # ========================================
    
    print(f"\n{'='*80}")
    print("GENERANDO ARCHIVOS DE SALIDA")
    print(f"{'='*80}")
    
    # Seleccionar columnas relevantes
    columnas_salida = ['id_participante']
    
    # Agregar demográficas si existen
    for col in ['timestamp', 'edad', 'genero', 'ano_academico', 'actividades_extra', 'miembro_wa']:
        if col in df.columns:
            columnas_salida.append(col)
    
    # Agregar puntuaciones psicométricas
    columnas_salida.extend([
        'ucla_total', 'ucla_categoria',
        'dass_depresion', 'dass_ansiedad', 'dass_estres',
        'dass_depresion_cat', 'dass_ansiedad_cat', 'dass_estres_cat'
    ])
    
    # Filtrar solo columnas que existen
    columnas_salida = [col for col in columnas_salida if col in df.columns]
    
    df_psicometrico = df[columnas_salida].copy()
    
    # Ordenar por id_participante (numérico si es posible) antes de guardar
    if 'id_participante' in df_psicometrico.columns:
        try:
            nums = df_psicometrico['id_participante'].astype(str).str.extract(r'(\d+)$')[0]
            if nums.notna().any():
                df_psicometrico['_orden'] = nums.astype(float).astype('Int64')
                df_psicometrico = df_psicometrico.sort_values(['_orden', 'id_participante']).drop(columns=['_orden'])
            else:
                df_psicometrico = df_psicometrico.sort_values('id_participante')
        except Exception:
            df_psicometrico = df_psicometrico.sort_values('id_participante')

    # Guardar archivo principal
    archivo_salida = os.path.join(CARPETA_SALIDA, 'datos_psicometricos.csv')
    df_psicometrico.to_csv(archivo_salida, index=False, encoding='utf-8-sig')
    print(f"\n✓ {archivo_salida}")
    print(f"  Total: {len(df_psicometrico)} participantes")
    print(f"  Columnas: {len(df_psicometrico.columns)}")
    
    # Guardar archivo con todos los detalles (incluyendo respuestas individuales)
    archivo_completo = os.path.join(CARPETA_SALIDA, 'datos_psicometricos_completo.csv')

    # Ordenar df completo por id_participante antes de guardar
    if 'id_participante' in df.columns:
        try:
            nums_all = df['id_participante'].astype(str).str.extract(r'(\d+)$')[0]
            if nums_all.notna().any():
                df['_orden'] = nums_all.astype(float).astype('Int64')
                df = df.sort_values(['_orden', 'id_participante']).drop(columns=['_orden'])
            else:
                df = df.sort_values('id_participante')
        except Exception:
            df = df.sort_values('id_participante')

    df.to_csv(archivo_completo, index=False, encoding='utf-8-sig')
    print(f"\n✓ {archivo_completo}")
    print(f"  Incluye todas las respuestas individuales")
    
    # Vista previa
    print(f"\n{'='*80}")
    print("VISTA PREVIA")
    print(f"{'='*80}")
    cols_preview = ['id_participante', 'ucla_total', 'dass_depresion', 'dass_ansiedad', 'dass_estres']
    cols_preview = [c for c in cols_preview if c in df_psicometrico.columns]
    print(df_psicometrico[cols_preview].head().to_string(index=False))
    
    print(f"\n{'='*80}")
    print("✓ PROCESAMIENTO COMPLETADO")
    print(f"{'='*80}")
    
    return df_psicometrico



if __name__ == "__main__":
    df_resultado = procesar_formulario_psicometrico()